In [1]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 

c:\programmi\python27\lib\site-packages\larlib\larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


0.0205706355483 seconds


%-- 1 cubito = 444 mm ----------------------------------------%
DEF r0 = 24; %---cubiti; raggio interno del muro interno------%
DEF r2 = 48; %---cubiti; raggio esterno del muro medio--------%
DEF r3 = 72; %---cubiti; raggio esterno del muro esterno------%
DEF r4 = 88; %---cubiti; raggio esterno scale esterne---------%

DEF wsteps = 10;  %---cubiti; profondita' scale esterne-------%
DEF nsteps = 10;  %---numero alzate/pedate scale esterne------%
DEF wstep = wsteps/nsteps; %---cubiti; larghezza pedata-------%
DEF hstep = 1/3; %---cubiti; altezza alzata-------------------%
DEF hw4 = 29; %--cubiti; altezza all'imposta del muro esterno-%
DEF hbasament = hstep*nsteps;

In [2]:
r0=24.00
r2 = 48.00
r3 = 72.00
r4 = 88.00
wsteps = 10.00
nsteps = 10.00
wstep = wsteps/nsteps
hstep = 1.00/3.00
hw4 = 29.00
hbasement = hstep*nsteps

DEF cylMap = MAP:[S2 * sin ~ S1, S2 * cos ~ S1, S3];

In [3]:
fx = lambda x: x[1] * math.cos(x[0])
fy = lambda x : x[1] * math.sin(x[0])
fz = lambda x : x[2]

cylMap = MAP([fx,fy,fz])

DEF vdom (h::IsRealPos) = ((EMBED:1 ~ Intervals:(PI*3/24)):3) * QUOTE:<h>;
DEF hdom (w::IsRealPos) = (S:2:-1 ~ EMBED:1):(Intervals:(PI*3/24):3 * QUOTE:<w>);

DEF steps (w,h::IsRealPos) (n::IsIntPos) = (STRUCT ~ CAT ~ #:n):
  < vdom:h, T:3:h, hdom:w, T:2:(-:w) >;

DEF hbasament = hstep*nsteps;
DEF stair = (MKPOL ~ UKPOL):(steps:<wstep,hstep>:nsteps  
              RIGHT (S:2:-1 ~ @2 ~ CUBOID):<PI/24,wsteps,hbasament>);
DEF ramp  = cylMap:(T:2:r4:stair);
DEF stairs = (STRUCT ~ ##:12):<ramp, R:<1,2>:(PI/6)>;

In [4]:
def vdom (h):
    return PROD([COMP([EMBED(1),INTERVALS(PI*3/24)])(3),QUOTE([h])])
def hdom (w):
    return COMP([S(2)(-1),EMBED(1)])(PROD([INTERVALS(PI*3/24)(3),QUOTE([w])]))
def steps (w,h,n):
    return COMP([STRUCT,CAT,N(n)])([vdom(h),T(3)(h),hdom(w),T(2)(-w)])
hbasament = hstep * nsteps
stair = RIGHT([COMP([MKPOL,UKPOL])(steps(wstep,hstep,nsteps)),COMP([S(2)(-1),SKEL_2,CUBOID])([PI/24,wsteps,hbasament])])
ramp = cylMap(T(2)(r4)(stair))
stairs = STRUCT(NN(12)([ramp,R([1,2])(PI/6)]))

%-------------------------------------------------------------%
%---Basamento-------------------------------------------------%
%-------------------------------------------------------------%

DEF basisSector = (cylMap ~ EMBED:1):
    (Intervals:(2*PI/12):4 * Intervals:(r4 - (wstep*(nsteps - 1))):1);
DEF basis = (STRUCT ~ ##:12):<basisSector, R:<1,2>:(2*PI/12)>;
DEF basement = (R:<1,2>:(PI/-48) ~ STRUCT):< stairs, T:3:(hstep*nsteps), basis >;

In [5]:
basi = PROD([INTERVALS(2*PI/12)(4),INTERVALS(r4 - (wstep*(nsteps-1)))(1)])
basisSector = COMP([cylMap,EMBED(1)])(basi)
basis = COMP([STRUCT,NN(12)])([basisSector,R([1,2])(2*PI/12)])
basement = COMP([R([1,2])(PI/-48),STRUCT])([stairs,T(3)(hstep*nsteps),basis])
VIEW(basement)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001445D450> >

![Basamento_1.PNG](files/Immagini/Basamento_1.PNG)

![Basamento_2.PNG](files/Immagini/Basamento_2.PNG)

In [6]:
fx = lambda x: x[1] * math.sin(x[0])
fy = lambda x : x[1] * math.cos(x[0])
fz = lambda x : x[2]

cylMap = MAP([fx,fy,fz])

%-------------------------------------------------------------%
%---Muro esterno----------------------------------------------%
%-------------------------------------------------------------%

DEF ExtWall2Da = MKPOL:<<<0,0>,<7,0>,<7,5>,<0,8>,<7,3>,<9,4>,
<10.5,1.5>,<10.5,0>,<11,1.5>,<11,3>>,<1..4,<2,8,7,6,5>,<6,7,9,10>>,
<<1>,<2>,<3>>> & ((Q ~ #:12):1 * Q:8);
DEF ExtWall2Db = MKPOL:<<<0.5,0>,<4,0>,<5,0>,<6,0>,<6,3>,<5.5,3.5>,
  <5,3>,<4.5,3.5>,<4,3>,<2,4>,<0,3>,<0,1.5>,<0.5,1.5>>,
  <<10,11,12,13>,<1,2,9,10,13>,<2,3,7,8,9>,<3,4,5,6,7>>,<1..4>> 
&  ((Q ~ #:6):1 * Q:4);
DEF ExtWall2D = STRUCT:<ExtWall2Da, T:1:11, ExtWall2Db>;
DEF sizxExtWall = SIZE:1:ExtWall2D;

DEF ExtWall = R:<2,3>:(PI/2):(ExtWall2D * QUOTE:<1.5>);
DEF CurvedExtWall = (cylMap ~ T:2:r3 ~ S:<1,3>:<PI/(4*sizxExtWall),hw4/8>):   ExtWall;
DEF DoubleExtWall = STRUCT:<CurvedExtWall, S:1:-1, CurvedExtWall>;
DEF FullExtWall = (STRUCT ~ ##:4):<DoubleExtWall, R:<1,2>:(PI/2)>;

%
STRUCT:<basement, T:3:hbasament, FullExtWall>;
%

In [7]:
ExtWall2Da = INTERSECTION([MKPOL([[[0,0],[7,0],[7,5],
                                   [0,8],[7,3],[9,4],
                                   [10.5,1.5],[10.5,0],
                                   [11,1.5],[11,3]],[range(1,5),[2,8,7,6,5],[6,7,9,10]],[[1],[2],[3]]]),PROD([COMP([QUOTE,N(12)])(1),Q(8)]) ])
ExtWall2Db = INTERSECTION ([
    MKPOL([[[0.5,0],[4,0],[5,0],[6,0],[6,3],[5.5,3.5],[5,3],[4.5,3.5],[4,3],[2,4],[0,3],[0,1.5],[0.5,1.5]],[[10,11,12,13],[1,2,9,10,13],[2,3,7,8,9],[3,4,5,6,7]],range(1,5)]),PROD([COMP([QUOTE,N(6)])(1),Q(4)])
])

ExtWall2D = STRUCT([ExtWall2Da,T(1)(11),ExtWall2Db])
sizxExtWall = SIZE(1)(ExtWall2D)
ExtWall = R([2,3])(PI/2)(PROD([ExtWall2D,QUOTE([1,5])]))
CurvedExtWall = COMP([cylMap,T(2)(r3),S([1,3])([PI/(4*sizxExtWall),hw4/8])])(ExtWall)
DoubleExtWall = STRUCT([CurvedExtWall,S(1)(-1),CurvedExtWall])
FullExtWall = COMP([STRUCT,NN(4)])([DoubleExtWall,R([1,2])(PI/2)])
VIEW(FullExtWall)
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001445DF00> >

![Muro_esterno_1.PNG](files/Immagini/Muro_esterno_1.PNG)

![Composizione_1.PNG](files/Immagini/:Composizione_1.PNG)

%-------------------------------------------------------------%
%---Colonne intermedie----------------------------------------%
%-------------------------------------------------------------%

DEF Column (w,h::IsRealPos) = basis TOP trunk TOP capitel
WHERE
  basis = CUBOID:<w,w,2*w/3>,
  trunk = CYLINDER:<w/2*0.85, h - w>:8,
  capitel = CUBOID:<w,w,w/3>
END;
DEF arcAngle = 2*PI/50.4;
DEF wallAngle = -3.2*arcAngle/4;
DEF RotCross = 3.2*arcAngle/4 + 2.5*arcAngle;

In [8]:
def Column(w,h):
    basis = CUBOID([w,w,2*w/3])
    trunk = CYLINDER ([w/2*0.85, h - w])(8)
    capitel = CUBOID([w,w,w/3])
    return TOP([capitel,TOP([trunk,basis])])

arcAngle = 2*PI/50.4
wallAngle = -3.2*arcAngle/4
RotCross = 3.2*arcAngle/4 + 2.5*arcAngle

DEF MyColumn = (MKPOL ~ UKPOL ~ T:2:((r2 - 0.75)) ~ Column):<1.5,hCol>;
DEF The4cols = (R:<1,2>:(arcAngle*0.4/3.2) ~ MKPOL ~ UKPOL ~ 
  STRUCT ~ ##:4):< R:<1,2>:(-:arcAngle), MyColumn >;
DEF The5cols = (R:<1,2>:(arcAngle*0.4/3.2) ~ MKPOL ~ UKPOL ~ 
  STRUCT ~ ##:5):< R:<1,2>:(-:arcAngle), MyColumn >;
DEF TheBotWal = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ CUBOID):
  < 3.2*arcAngle/4, 1.5, hCol >;
DEF TheSecCols = STRUCT:< R:<1,2>:RotCross,
  TheBotWal, R:<1,2>:wallAngle, 
  The4cols, R:<1,2>:(-5*arcAngle), 
  TheBotWal, R:<1,2>:wallAngle, 
  The5cols >;
DEF TheMedColumns = (STRUCT ~ ##:4):<TheSecCols, R:<1,2>:(PI/-2)>;
DEF hCol = 12 ;


In [9]:
hCol = 12
def Column(argomenti):
    w,h = argomenti
    basis = CUBOID([w,w,2*w/3])
    trunk = CYLINDER ([w/2*0.85, h - w])(8)
    capitel = CUBOID([w,w,w/3])
    return TOP([TOP([trunk,basis]),capitel])

arcAngle = 2*PI/50.4
wallAngle = -3.2*arcAngle/4
RotCross = 3.2*arcAngle/4 + 2.5*arcAngle

MyColumn  = COMP([MKPOL,UKPOL,T(2)(r2 - 0.75),Column])([1.5,hCol])
The4cols = COMP([R([1, 2])(arcAngle*.4/3.2), MKPOL, UKPOL, STRUCT, NN(4)])([R([1, 2])(-arcAngle), MyColumn])
The5cols = COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(5)])([R([1,2])(-arcAngle),MyColumn])
TheBotWal = COMP([cylMap,MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5, hCol])
TheSecCols = STRUCT([R([1,2])(RotCross),TheBotWal,R([1,2])(wallAngle),The4cols,R([1,2])(-5*arcAngle),TheBotWal,R([1,2])(wallAngle),The5cols])
TheMedColumns = COMP([STRUCT,NN(4)])([TheSecCols,R([1,2])(PI/(-2))])

VIEW(TheMedColumns)



<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145E12A0> >

![Colonne_intermedie_1.PNG](files/Immagini/Colonne_intermedie_1.PNG)

In [10]:
#%
#STRUCT:<basement, T:3:hbasament, FullExtWall, TheMedColumns >;
#%
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,TheMedColumns]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000014529F90> >

![Composizione_2.PNG](files/Immagini/Composizione_2.PNG)

%-------------------------------------------------------------%
%---Archi sulle colonne---------------------------------------%
%-------------------------------------------------------------%

DEF bottomArc (d::IsRealPos) = Bezier:S1:<<0,0>,<0,2*d/3>,<d,2*d/3>,<d,0>>;
DEF topArc (d::IsRealPos) = Bezier:S1:<<0,2*d/3>,<d,2*d/3>>;
DEF arc2D  (d::IsRealPos) = Bezier:S2:<bottomArc:d,topArc:d>;
DEF arc3D  (d::IsRealPos)(w::IsRealPos) = (T:2:w ~ R:<2,3>:(PI/2)):
  ( MAP:(CONS:(arc2D:d)):(Intervals:1:8 * Intervals:1:1) * QUOTE:<w> );
DEF Interarc (d1,d2::IsReal)(w::IsRealPos)= CUBOID:<d1,w,2*d2/3>;
DEF Xarc (d1,d2::IsRealPos) (w::IsRealPos) = 
Interarc:<d1,d2>:w RIGHT arc3D:d2:w RIGHT Interarc:<d1,d2>:w;

DEF SizeArc = SIZE:1:TheArc;
DEF TheArc = Xarc:<0.4,3.2>:1.5;
DEF HeigthArc = SIZE:3:CurvedArc;
DEF CurvedArc = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ S:<1,3>:<sx,sz>):TheArc
WHERE
  sx = (1/SizeArc)*arcAngle,
  sz = sx * (r2 - 2)
END;
DEF The5arcs = (MKPOL ~ UKPOL ~ STRUCT ~ ##:5):< CurvedArc, R:<1,2>:(-:arcAngle) >;
DEF The6arcs = (MKPOL ~ UKPOL ~ STRUCT ~ ##:6):< CurvedArc, R:<1,2>:(-:arcAngle) >;
DEF TheMidWall = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ CUBOID):
  <3.2*arcAngle/4, 1.5, HeigthArc>;
DEF TheSector = STRUCT:< R:<1,2>:RotCross,
  TheMidWall, R:<1,2>:wallAngle, 
  The5arcs, R:<1,2>:(-5*arcAngle), 
  TheMidWall, R:<1,2>:wallAngle, 
  The6arcs >;
DEF TheArcs = (STRUCT ~ ##:4):<TheSector, R:<1,2>:(PI/-2)>;
DEF TheArcsCols = STRUCT:< TheMedColumns, T:3:hCol, TheArcs >;

In [11]:
def bottomArv(d):
    return BEZIER(S1)([[0,0],[0,2*d/3],[d,2*d/3],[d,0]])

def topArc (d) :
    return BEZIER (S1)([[0,2*d/3],[d,2*d/3]])
def arc2D (d) :
    return BEZIER(S2)([bottomArv(d),topArc(d)])

def arc3D (d) :
    def arc3D1(w):
        return COMP([T(2)(w),R([2,3])(PI/2)])(PROD([MAP(arc2D(d))(PROD([INTERVALS(1)(8),INTERVALS(1)(1)])),QUOTE([w])]))
    return arc3D1

def Interarc (d1,d2):
    def Interarc1(w):
        return CUBOID([d1,w,2*d2/3])
    return Interarc1

def Xarc(d1,d2):
    def Xarc1(w):
        return RIGHT([RIGHT([Interarc(d1,d2)(w),arc3D(d2)(w)]),Interarc(d1,d2)(w)])
    return Xarc1

def CurvedArc():
    sx = (1/SizeArc)* arcAngle
    sz = sx * (r2-2)
    return COMP([cylMap,MKPOL,UKPOL,T(2)(r2 - 0.75),S([1,3])([sx,sz])])(TheArc)

TheArc = Xarc(0.4,3.2)(1.5)
SizeArc = SIZE(1)(TheArc)
HeightArc = SIZE(3)(CurvedArc())
The5arcs = COMP([MKPOL,UKPOL,STRUCT,NN(5)])([CurvedArc(),R([1,2])(-arcAngle)])
The6arcs = COMP([MKPOL,UKPOL,STRUCT,NN(6)])([CurvedArc(),R([1,2])(-arcAngle)])
TheMidWall = COMP([cylMap,MKPOL,UKPOL,T(2)(r2 - 0.75),CUBOID])([3.2*arcAngle/4,1.5,HeightArc])
TheSector = STRUCT([R([1,2])(RotCross),TheMidWall,R([1,2])(wallAngle),The5arcs,R([1,2])(-(5*arcAngle)),TheMidWall,R([1,2])(wallAngle),The6arcs])
TheArcs = COMP([STRUCT,NN(4)])([TheSector,R([1,2])(PI/-2)])
TheArcsCols = STRUCT([TheMedColumns,T(3)(hCol),TheArcs])

VIEW(TheArcsCols)
        



<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145E1E40> >

![Archi_sulle_colonne.PNG](files/Immagini/Archi_sulle_colonne.PNG)


%
STRUCT:<basement, T:3:hbasament, FullExtWall, TheArcsCols >;
%

In [12]:
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,TheArcsCols]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000014529F60> >

![Composizione_3.PNG](files/Immagini/Composizione_3.PNG)

DEF MedWall2D = MKPOL:<<<0,4>,<8,4>,<17,4>,<17,5>,<12.5,7.5>,<8,5>,<0,8>>,
<<1,2,6,7>,<2,3,4,5,6>>,
<<1>,<2>>> & (QUOTE:(#:17:1) * QUOTE:<8>);
DEF sizxMedWall = SIZE:1:MedWall2D;

DEF MedWall = R:<2,3>:(PI/2):(MedWall2D * QUOTE:<1.5>);
DEF CurvedMedWall = (cylMap ~ MKPOL ~ UKPOL ~ 
    T:2:(r2+0.75) ~ S:<1,3>:<PI/(4*sizxMedWall),hw4/8>):MedWall;

DEF DoubleMedWall = STRUCT:<CurvedMedWall, S:1:-1:CurvedMedWall>;
DEF FullMedWall = (STRUCT ~ ##:4):<DoubleMedWall, R:<1,2>:(PI/2)>;
DEF MedWallArcs = STRUCT:< FullMedWall, TheArcsCols >;

In [13]:
MedWall2D = INTERSECTION([MKPOL([[[0,4],[8,4],[17,4],[17,5],[12.5,7.5],[8,5],[0,8]],[[1,2,6,7],[2,3,4,5,6]],[[1],[2]]]),PROD([COMP([QUOTE,N(17)])(1),QUOTE([8])])])

sizxMedWall = SIZE(1)(MedWall2D)
MedWall = R([2,3])(PI/2) (PROD([MedWall2D,QUOTE([1.5])]))
CurvedMedWall = COMP([cylMap, MKPOL,UKPOL, T(2)(r2 + 0.75),S([1,3])([PI/(4*sizxMedWall),hw4/8])])(MedWall)
DoubleMedWall = STRUCT([CurvedMedWall,S(1)(-1),CurvedMedWall])
FullMedWall = COMP([STRUCT,NN(4)])([DoubleMedWall, R([1,2])(PI/2)])
MedWallArcs = STRUCT([FullMedWall,TheArcsCols])

%
STRUCT:<basement, T:3:hbasament, FullExtWall, MedWallArcs >;
%

In [14]:
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,MedWallArcs]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145E1EA0> >

![Composizione_4.PNG](files/Immagini/Composizione_4.PNG)

%-------------------------------------------------------------%
%---Muri laterali della croce---------------------------------%
%-------------------------------------------------------------%


DEF CrossWall = (T:1:-1 ~ R:<1,2>:(0.2*arcAngle) ~ 
  S:3:(hw4/8) ~ CUBOID):<1.5,r3 - r2,5>;
DEF SingleWall =  (R:<1,2>:(2.5*arcAngle) ~ T:2:r2):CrossWall;
DEF DoubleWall =  STRUCT:< SingleWall, S:1:-1, SingleWall >;
DEF CrossWalls = (STRUCT ~ ##:4):<DoubleWall, R:<1,2>:(PI/2)>;

%
STRUCT:<basement, T:3:hbasament, FullExtWall, MedWallArcs, CrossWalls >;
%

In [15]:
CrossWall = COMP([T(1)(-1),R([1,2])(0.2*arcAngle),S(3)(hw4/8),CUBOID])([1.5,r3-r2,5])
SingleWall = COMP([R([1,2])(2.5*arcAngle),T(2)(r2)])(CrossWall)
DoubleWall = STRUCT([SingleWall,S(1)(-1),SingleWall])
CrossWalls = COMP([STRUCT,NN(4)])([DoubleWall,R([1,2])(PI/2)])
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall, MedWallArcs,CrossWalls]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145F4840> >

![Composizione_5.PNG](files/Immagini/Composizione_5.PNG)

%-------------------------------------------------------------%
%---Tetto sulla croce-----------------------------------------%
%-------------------------------------------------------------%

DEF HalfTetto = MKPOL:<<<0,r2,8>,<8,r2,5>,<8,r3,5>,<0,r3,8>>,<1..4>,<<1>>>;
DEF CurvedHalfTetto = (cylMap ~ S:<1,3>:<2.5*arcAngle/8,hw4/8>):HalfTetto;
DEF DoubleHalfTetto = STRUCT:<CurvedHalfTetto, S:1:-1:CurvedHalfTetto>;
DEF FullTetto = (STRUCT ~ ##:4):<DoubleHalfTetto, R:<1,2>:(PI/-2)>;

%
STRUCT:<basement, T:3:hbasament, FullExtWall, MedWallArcs, 
        CrossWalls, FullTetto >;
%

In [16]:
HalfTetto = MKPOL([[[0,r2,8],[8,r2,5],[8,r3,5],[0,r3,8]],[range(1,5)],[[1]]])
CurvedHalfTetto = COMP([cylMap,S([1,3])([2.5*arcAngle/8,hw4/8])])(HalfTetto)
DoubleHalfTetto = STRUCT([CurvedHalfTetto,S(1)(-1)(CurvedHalfTetto)])
FullTetto = COMP([STRUCT,NN(4)])([DoubleHalfTetto,R([1,2])(PI/-2)])
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,MedWallArcs,CrossWalls,FullTetto]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145F4B10> >

![Composizione_6.PNG](files/Immagini/Composizione_6.PNG)

%-------------------------------------------------------------%
%---Muro con Triplice apertura--------------------------------%
%-------------------------------------------------------------%

DEF CourtWall (d1,d2::IsRealPos) (n1,n2::IsIntPos) = 
  LeftWall OP TripleHole OP RightWall
WHERE
  op = ALIGN:<<1,MAX,MIN>,<2,MIN,MIN>>,
  LeftWall = ((QUOTE ~ #:n1):(d1/n1)) * CUBOID:<1,h>,
  RightWall = ((QUOTE ~ #:n2):(d2/n2)) * CUBOID:<1,h>,
  h = SIZE:3:TripleHole,
  TripleHole = STRUCT:<column:<w,2>, T:1:(2+w), column:<w,2>> 
    TOP Xarc:<2,2>:1,
  w = 0.7
END;
%
CourtWall:<6,6>:<3,3>
%


In [17]:
def CourtWall(d1,d2):
    def CourtWall1(n1,n2):
        w = 0.7
        TripleHole = TOP([STRUCT([Column([w,2]),T(1)(2+w),Column([w,2])]),Xarc(2,2)(1)])
        h = SIZE(3)(TripleHole)
        op = ALIGN([[1,MAX,MIN],[2,MIN,MIN]])
        LeftWall = PROD([COMP([QUOTE,N(n1)])(d1/n1),CUBOID([1,h])])
        RightWall = PROD([COMP([QUOTE,N(n2)])(d2/n2),CUBOID([1,h])])
        return op([LeftWall,op([TripleHole,RightWall])])
        #return LeftWall OP TripleHole OP RightWall
    return CourtWall1

VIEW(CourtWall(6,6)(3,3))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145F4F60> >

![Muro_con_Triplice_apertura.PNG](files/Immagini/Muro_con_Triplice_apertura.PNG)

%-------------------------------------------------------------%
%---Muro inferiore Cortile interno----------------------------%
%-------------------------------------------------------------%

DEF MyCourtWall = CourtWall:<12,12>:<9,9>;
DEF sizxCourtWall = SIZE:1:MyCourtWall;
DEF CurvedCourtWall = (optimize ~ 
    R:<1,2>:(-2.5*arcAngle) ~ mapping ~ T:2:((r3+r2)/2)): MyCourtWall;
DEF mapping = (cylMap ~ S:<1,3>:<(7.25*PI/24)*(1/sizxCourtWall),hw4/8>);
DEF CrossCourtWall = (STRUCT ~ ##:4):<CurvedCourtWall, R:<1,2>:(PI/2)>;
%
STRUCT:<FullTetto, CrossWalls, CrossCourtWall >;
STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns>;
%

In [18]:
MyCourtWall = CourtWall(12,12)(9,9)
sizxCourtWall = SIZE(1)(MyCourtWall)
mapping = COMP([cylMap,S([1,3])([(7.25*PI/24)*(1/sizxCourtWall),hw4/8])])

CurvedCourtWall = COMP([OPTIMIZE,R([1,2])(-2.5*arcAngle),mapping,T(2)((r2+r2)/2)])(MyCourtWall)
CrossCourtWall = COMP([STRUCT,NN(4)])([CurvedCourtWall,R([1,2])(PI/2)])
VIEW(STRUCT([FullTetto,CrossWalls,CrossCourtWall]))
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,CrossWalls,MedWallArcs,TheMedColumns]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145F4FC0> >

![Muro_inferiore_Cortile_interno.PNG](files/Immagini/Muro_inferiore_Cortile_interno.PNG)

![Composizione_7.PNG](files/Immagini/Composizione_7.PNG)


%-------------------------------------------------------------%
%---Coronamento Muro Cortile interno--------------------------%
%-------------------------------------------------------------%



DEF TopCourtWall2D = MKPOL:<<<0,3.333>,<10,3.333>,<10,4.333>,<5,6.333>,<0,4.333>>,
    <1..5>,<<1>>> & (QUOTE:(#:10:1) * QUOTE:<8>);
DEF TopCourtWall = R:<2,3>:(PI/2):(TopCourtWall2D * QUOTE:<1>);
DEF MyTopCourtWall = TopCourtWall RIGHT TopCourtWall RIGHT TopCourtWall; 
DEF CurvedTopCourtWall = (
    R:<1,2>:(-2.5*arcAngle) ~ mapping ~ T:2:((r3+r2+2)/2)): MyTopCourtWall;
DEF FullTopCourtWall = (STRUCT ~ ##:4):<CurvedTopCourtWall, R:<1,2>:(PI/2)>;
%
STRUCT:<FullTetto, CrossWalls, CrossCourtWall, FullTopCourtWall >;

STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall>;
%

In [19]:
TopCourtWall2D = INTERSECTION([MKPOL([[[0,3.333],[10,3.333],[10,4.333],[5,6.333],[0,4.333]],[range(1,6)],[[1]]]),PROD([COMP([QUOTE,N(10)])(1),QUOTE([8])])])
TopCourtWall = R([2,3])(PI/2)(PROD([TopCourtWall2D,QUOTE([1])]))
MyTopCourtWall = RIGHT([TopCourtWall,RIGHT([TopCourtWall,TopCourtWall])])
CurvedTopCourtWall = COMP([R([1,2])(-2.5*arcAngle),mapping,T(2)((r3+r2+2)/2)])(MyTopCourtWall)
FullTopCourtWall = COMP([STRUCT,NN(4)])([CurvedTopCourtWall,R([1,2])(PI/2)])
VIEW(STRUCT([FullTetto,CrossWalls,CrossCourtWall,FullTopCourtWall]))
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,CrossWalls,MedWallArcs,TheMedColumns,FullTopCourtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000014517030> >

![Conoramento_Muro_Cortile_interno.PNG](files/Immagini/Conoramento_Muro_Cortile_interno.PNG)

![Composizione_8.PNG](files/Immagini/Composizione_8.PNG)

%-------------------------------------------------------------%
%---Tetto su Cortile interno----------------------------------%
%-------------------------------------------------------------%

DEF RotWall = 2.5*arcAngle;
DEF CourTetto = MKPOL:<<<0,(r3+r2+2)/2,3.333>,<10,(r3+r2+2)/2,3.333>,
    <10,r2,4.333>,<0,r2,4.333>,
    <0,(r3+r2+2)/2,3.433>,<10,(r3+r2+2)/2,3.433>,
    <10,r2,4.433>,<0,r2,4.433> >, <1..8>,<<1>>> 
& 
   (QUOTE:(#:10:1) * QUOTE:<(r3+r2+2)/2> * QUOTE:<5>);

DEF MyCourTetto = CourTetto RIGHT CourTetto RIGHT CourTetto; 
DEF CurvedCourTetto = (R:<1,2>:(-:RotWall) ~ mapping): MyCourTetto;
DEF FullCourTetto = (STRUCT ~ ##:4):<CurvedCourTetto, R:<1,2>:(PI/2)>;

%
STRUCT:<FullTetto, CrossWalls, CrossCourtWall, FullTopCourtWall,
        FullCourTetto >;
STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, FullCourTetto>;
%


In [20]:
RotWall = 2.5 * arcAngle
CourTetto = INTERSECTION([MKPOL([[[0,(r3+r2+2)/2,3.333],[10,(r3+r2+2)/2,3.333],[10,r2,4.333],[0,r2,4.333],[0,(r3+r2+2)/2,3.433],[10,(r3+r2+2)/2,3.433],[10,r2,4.433],[0,r2,4.433]],[range(1,9)],[[1]]]),
PROD([COMP([QUOTE,N(10)])(1),PROD([QUOTE([(r3+r2+2)/2]),QUOTE([5])])])])
MyCourTetto = RIGHT([CourTetto,RIGHT([CourTetto,CourTetto])])
CurvedCourTetto = COMP ([R([1,2])(-(RotWall)),mapping])(MyCourTetto)
FullCourTetto = COMP([STRUCT,NN(4)])([CurvedCourTetto,R([1,2])(PI/2)])
VIEW(STRUCT([FullTetto,CrossWalls,CrossCourtWall,FullTopCourtWall,FullCourTetto]))
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,CrossWalls,MedWallArcs,TheMedColumns,FullTopCourtWall,FullCourTetto]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145173C0> >

![Tetto_su_Cortile_interno.PNG](files/Immagini/Tetto_su_Cortile_interno.PNG)

![Composizione_9.PNG](files/Immagini/Composizione_9.PNG)

%-------------------------------------------------------------%
%---Colonne muro interno--------------------------------------%
%-------------------------------------------------------------%

DEF Radial22Obj (Obj::IsPol) = 
  STRUCT:< The20Obj, R:<1,2>:((20*PI)/11), The2Obj >
WHERE
  The4Obj = (STRUCT ~ ##:4):< Obj, R:<1,2>:(PI/11) >,
  The2Obj  = (STRUCT ~ ##:2):< Obj, R:<1,2>:(PI/11) >,
  The20Obj = (STRUCT ~ ##:5):< The4Obj, R:<1,2>:((4*PI)/11) >
END;

DEF InternalCol = (MKPOL ~ UKPOL ~ T:2:((r0 - 2)) ~ Column):<1.5,hCol>;
DEF The22Columns = Radial22Obj:InternalCol;


In [21]:
def Radial22Obj(Obj):
    The4Obj = COMP([STRUCT,NN(4)])([Obj,R([1,2])(PI/11)])
    The2Obj = COMP([STRUCT,NN(2)])([Obj,R([1,2])(PI/11)])
    The20Obj = COMP([STRUCT,NN(5)])([The4Obj,R([1,2])((4*PI)/11)])
    return STRUCT([The20Obj,R([1,2])((20*PI)/11),The2Obj])

InternalCol = COMP([MKPOL,UKPOL,T(2)((r0-2)),Column])([1.5,hCol])
The22Columns = Radial22Obj(InternalCol)

%-------------------------------------------------------------%
%---Travi muro interno----------------------------------------%
%-------------------------------------------------------------%

DEF Beam = Q:<-0.5,1.5,2,1.5> * Q:<2.2> * Q:1.5;
DEF CurvedBeam = (cylMap ~ T:<2,3>:<(r0 - 2),hcol> ~ S:1:(PI/(11*5)) ):Beam;
DEF The22Beams = Radial22Obj:CurvedBeam;

%
STRUCT:< The22Columns, The22Beams >;

STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
        FullCourTetto, The22Columns, The22Beams >;
%


In [22]:
Beam = PROD([QUOTE([-0.5,1.5,2,1.5]),PROD([QUOTE([2.2]),QUOTE([1.5])])])
CurvedBeam = COMP([cylMap,T([2,3])([(r0 - 2),hCol]),S(1)(PI/(11*5))])(Beam)
The22Beams = Radial22Obj(CurvedBeam)


VIEW(STRUCT([The22Columns,The22Beams]))
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,CrossWalls,MedWallArcs,TheMedColumns,FullTopCourtWall,FullCourTetto,The22Columns,The22Beams]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145177E0> >

![Travi_muro_interno.PNG](files/Immagini/Travi_muro_interno.PNG)

![Composizione_10.PNG](files/Immagini/Composizione_10.PNG)

%-------------------------------------------------------------%
%---Muro interno superiore------------------------------------%
%-------------------------------------------------------------%

DEF Hwall = STRUCT:<
%  QUOTE:<-0.5,1.5,-2,1.5> * QUOTE:<-0.3,1.4> * QUOTE:<2.5*hCol>,
  QUOTE:<-2.0,2> * QUOTE:<-0.3,1.4> * QUOTE:<1.5*hCol, -0.5*hcol, 0.5*hcol>
%  QUOTE:<-0.5,1.75,-1.5,1.75> * QUOTE:<-0.3,1.4> * QUOTE:<2.5*hCol>,
  QUOTE:<-2.25,1.5> * QUOTE:<-0.3,1.4> * QUOTE:<1.5*hCol, -0.5*hcol, 0.5*hcol>
>;
DEF CurvedHwall = (cylMap ~ T:<2,3>:<(r0 - 2),hcol> ~ S:1:(PI/(11*5)) ):Hwall;
DEF The22Hwall = Radial22Obj:CurvedHwall;
%
STRUCT:< The22Columns, The22Beams, The22Hwall >;

STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
        FullCourTetto, The22Columns, The22Beams, The22Hwall >;
%

In [23]:
#Hwall = STRUCT([
     #PROD([QUOTE([-0.5,1.5,-2,1.5]),PROD([QUOTE([-0.3,1.4]),PROD([QUOTE([2.5*hCol]),PROD([QUOTE([-2,0.2]),PROD([QUOTE([-0.3,1.4]),PROD([QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol]),
    #PROD([QUOTE([-0.5,1.75,-1.5,1.75]),PROD([QUOTE([-0.3,1.4]),PROD([QUOTE([2.5*hCol]),PROD([QUOTE([-2.25,1.5]),PROD([QUOTE([-0.3,1.4]),QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol])])])])])])
#])])])])])
#])])
prova = INSR(PROD)([QUOTE([-2.0,2.0]),QUOTE([-0.3,1.4]),QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol])])
prova1 = INSR(PROD)([QUOTE([-2.25,1.5]),QUOTE([-0.3,1.4]),QUOTE([1.5*hCol, -0.5*hCol , 0.5 * hCol])])
prova2 = INSR(PROD)([QUOTE([-0.5,1.5,-2,1.5]), QUOTE([-0.3,1.4]), QUOTE([2.5*hCol])])


In [24]:
Hwall = STRUCT([prova,prova1,prova2])


In [25]:

CurvedHwall = COMP([cylMap,T([2,3])([(r0 - 2),hCol]),S(1)(PI/(11*5))])(Hwall)
The22Hwall = Radial22Obj(CurvedHwall)
VIEW(STRUCT([The22Columns,The22Beams,The22Hwall]))
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,CrossWalls,MedWallArcs,MedWallArcs,TheMedColumns,FullTopCourtWall,FullCourTetto,The22Columns,The22Beams,The22Hwall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000014517840> >

![Muro_interno_superiore.PNG](files/Immagini/Muro_interno_superiore.PNG)

![Composizione_11.PNG](files/Immagini/Composizione_11.PNG)

# %-------------------------------------------------------------%
%---Coronamento superiore muro interno -----------------------%
%-------------------------------------------------------------%

DEF myArc = MAP:[SIN ~ S1, COS ~ S1]:(QUOTE:(#:5:(PI/10)));
DEF Vertex = MKPOL:<<<0,0>>,<<1>>,<<1>>>;
DEF Sector = JOIN:<myArc, Vertex>;
DEF Triangle = S:1:-1:(SIMPLEX:2);
DEF SectorTriangle2D = STRUCT:<T:1:0.05, Sector, T:1:2, Triangle>;
DEF DoubleSect = STRUCT:< SectorTriangle2D, S:1:-1:SectorTriangle2D>;
DEF Sect2D = S:1:(2/2.05):DoubleSect;
DEF Sect3D = R:<2,3>:(PI/2):(Sect2D * QUOTE:<0.8>);
DEF CurvedSect = (cylMap ~ T:2:(r0 %- 2%) ~ S:<1,3>:<(2*PI)/(11*4),3> ):Sect3D;
DEF Frieze = T:3:(3.5*hcol):CurvedSect;
DEF Radial11Obj (Obj::IsPol) = 
  STRUCT:< The10Obj, R:<1,2>:((20*PI)/11), Obj >
WHERE
  The2Obj = (STRUCT ~ ##:2):< Obj, R:<1,2>:((2*PI)/11) >,
  The10Obj = (STRUCT ~ ##:5):< The2Obj, R:<1,2>:((4*PI)/11) >
END;
DEF The11Frieze = Radial11Obj:Frieze;

%
The11Frieze;
STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
        FullCourTetto, The22Columns, The22Beams, The22Hwall, The11Frieze >;
%

In [26]:
funz_x = lambda x : math.sin(x[0])
funz_y = lambda x : math.cos(x[0])
myArc = MAP([funz_x,funz_y])(COMP([QUOTE,N(5)])(PI/10))
Vertex = MKPOL([[[0,0]],[[1]],[[1]]])
Sector = JOIN([myArc,Vertex])
Triangle = S(1)(-1)((SIMPLEX(2)))
SectorTriangle2D = STRUCT([T(1)(0.05),Sector,T(1)(2),Triangle])
DoubleSect = STRUCT([SectorTriangle2D,S(1)(-1)(SectorTriangle2D)])
Sect2D = S(1)(2/2.05)(DoubleSect)
Sect3D = R([2,3])(PI/2)(PROD([Sect2D,QUOTE([0.8])]))
CurvedSect = COMP([cylMap, T(2)(r0),S([1,3])([(2*PI)/(11*4),3])])(Sect3D)
Frieze = T(3)(3.5*hCol)(CurvedSect)
def Radial11Obj(Obj):
    The2Obj = COMP([STRUCT,NN(2)])([Obj,R([1,2])((2*PI)/11)])
    The10Obj = COMP([STRUCT,NN(5)])([The2Obj,R([1,2])((4*PI)/11)])
    return STRUCT([The10Obj,R([1,2])((20*PI)/11),Obj])

The11Frieze = Radial11Obj(Frieze)
VIEW(The11Frieze)
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,
            CrossWalls,MedWallArcs,TheMedColumns,FullTopCourtWall,
            FullCourTetto,The22Columns,The22Beams,The22Hwall,The11Frieze]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000145178D0> >

![Coronamento_superiore_muro_interno.PNG](files/Immagini/Coronamento_superiore_muro_interno.PNG)

![Composizione_12.PNG](files/Immagini/Composizione_12.PNG)

%-------------------------------------------------------------%
%---Traliccio Tetto interno-----------------------------------%
%-------------------------------------------------------------%

DEF Trellis (H,L::IsRealPos) = (R:<2,3>:(PI/2) ~ EMBED:1 ~ MKPOL):
  <<<0,0>,<L/3,0>,<2*L/3,0>,<L,0>,<2*L/3,H/3>,<L/3,2*H/3>,<0,H>>,
   <<1,2>,<2,3>,<3,4>,<4,5>,<5,6>,<6,7>,<7,1>,<1,6>,<6,2>,<2,5>,<5,3>>,
   <1..11> > ;
DEF Radial (n::IsIntPos)(obj::IsPol) = (STRUCT ~ ##:3):
   <(STRUCT ~ ##:(n/3)):< obj, R:<1,2>:(2*PI/n) >, R:<1,2>:(2*PI/3)>;

DEF TrellisTop3D = (T:2:-0.25 ~ S:2:0.5 ~ OffSet:<1,1,1> ~ Trellis):<hcol,r0 - 1>;
DEF TheTopTrellis = T:3:(3.5*hcol):TrellisTop3D;
DEF TopTrellis = Radial:12:TheTopTrellis;
%
STRUCT:< The11Frieze, TopTrellis >;

STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
        FullCourTetto, The22Columns, The22Beams, The22Hwall, 
        The11Frieze, TopTrellis >;
%


In [27]:
def Trellis(argomenti):
    H,L  = argomenti
    return COMP([R([2,3])(PI/2),EMBED(1),MKPOL])([[[0,0],[L/3,0],[2*L/3,0],[L,0],[2*L/3,H/3],[L/3,2*H/3],[0,H]],[[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,1],[1,6],[6,2],[2,5],[5,3]],[range(1,12)]])
def Radial (n):
    def Radial1(obj):
        return COMP([STRUCT,NN(3)])(COMP([STRUCT,NN(n/3)])(obj,R([1,2])(2*PI/n)))(R([1,2])(2*PI/3))
    return Radial11Obj
TrellisTop3D = COMP([T(2)(-0.25),S(2)(0.5),OFFSET([1,1,1]),Trellis])([hCol,r0 -1])
TheTopTrellis = T(3)(3.5*hCol)(TrellisTop3D)
TopTrellis = Radial(12)(TheTopTrellis)
VIEW(STRUCT([The11Frieze,TopTrellis]))
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,
            CrossWalls,MedWallArcs,TheMedColumns,FullTopCourtWall,
            FullCourTetto,The22Columns,The22Beams,The22Hwall,
            The11Frieze,TopTrellis]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001464DA80> >

![Traliccio_Tetto_interno.PNG](files/Immagini/Traliccio_Tetto_interno.PNG)

![Composizione_13.PNG](files/Immagini/Composizione_13.PNG)

%-------------------------------------------------------------%
%---Traliccio Tetto intermedio--------------------------------%
%-------------------------------------------------------------%

DEF TrellisMed3D = (T:2:-0.25 ~ S:2:0.5 ~ OffSet:<1,1,1> ~ Trellis):<hcol,r2 - r0>;
DEF TheMedTrellis = T:3:(hcol+3):TrellisMed3D;
DEF MedTrellis = Radial:24:(T:1:r0:TheMedTrellis);
DEF DoubleMedTrellis = STRUCT:<MedTrellis, R:<1,2>:(PI/24), MedTrellis>;

In [28]:
TrellisMed3D = COMP([T(2)(-0.25),S(2)(0.5),OFFSET([1,1,1]),Trellis])([hCol,r2-r0])
TheMedTrellis = T(3)(hCol+3)(TrellisMed3D)

In [29]:
MedTrellis = Radial(24)(T(1)(r0)(TheMedTrellis))
DoubleMedTrellis = STRUCT([MedTrellis, R([1,2])(PI/24),MedTrellis])

%---------------------------------------------------------------------
DEF out = (CenteredCameras ~ STRUCT):
  <basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
        FullCourTetto, The22Columns, The22Beams, The22Hwall, 
        The11Frieze, TopTrellis , DoubleMedTrellis >;
---------------------------------------------------------------------%

DEF out = STRUCT:
  <basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
        FullCourTetto, The22Columns, The22Beams, The22Hwall, 
The11Frieze, TopTrellis , DoubleMedTrellis >;

In [30]:
out = STRUCT([basement,T(3)(hbasament),FullExtWall,FullTetto,CrossCourtWall,
                                      CrossWalls,MedWallArcs,TheMedColumns,FullTopCourtWall,
                                      FullCourTetto,The22Columns,The22Beams,The22Hwall,
                                      The11Frieze,TopTrellis,DoubleMedTrellis])

In [31]:
VIEW(out)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000014517720> >

![Composizione_finale.PNG](files/Immagini/Composizione_finale.PNG)